In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import pathlib
from Bio import SeqIO

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

from config import MATERIALS_PATH, RESULTS_PATH

FEATURE_PATH = MATERIALS_PATH.joinpath('mart_export.txt')

UNSPLICED_PATH = MATERIALS_PATH.joinpath('martquery_0219183013_823.txt')
UNSPLICED_RESULT_PATH = MATERIALS_PATH.joinpath('gene_symbol_dna_sequence_unspliced.pkl')

EXON_PATH = MATERIALS_PATH.joinpath('martquery_0225145730_334.txt')
EXON_RESULT_PATH = MATERIALS_PATH.joinpath('gene_symbol_dna_sequence_exon.pkl')

PROTEIN_FASTA_PATH = MATERIALS_PATH.joinpath('uniprot-compressed_true_download_true_format_fasta_includeIsoform_tr-2023.02.17-22.00.02.49.fasta')
PROTEIN_RESULT_PATH = MATERIALS_PATH.joinpath('gene_symbol_protein_sequences.pkl')

12


In [2]:
df = pd.read_csv(FEATURE_PATH, sep='\t')

df = df[['Gene stable ID', 'Gene name']]

df = df.drop_duplicates()

_df_features = df.copy()

df

,Gene stable ID,Gene name
0,ENSMUSG00000064336,mt-Tf
3,ENSMUSG00000064337,mt-Rnr1
7,ENSMUSG00000064338,mt-Tv
10,ENSMUSG00000064339,mt-Rnr2
14,ENSMUSG00000064340,mt-Tl1
18,ENSMUSG00000064341,mt-Nd1
22,ENSMUSG00000064342,mt-Ti
25,ENSMUSG00000064343,mt-Tq
28,ENSMUSG00000064344,mt-Tm
31,ENSMUSG00000064345,mt-Nd2


In [3]:
records = []
for datum in SeqIO.parse(UNSPLICED_PATH, "fasta"):
    record = {}
    record['Gene stable ID'] = datum.description
    record['Sequence'] = str(datum.seq)
    records.append(record)

df = pd.DataFrame(records)

df = _df_features.merge(df, how='outer')

assert not df['Gene stable ID'].duplicated().any()

df.to_pickle(UNSPLICED_RESULT_PATH)

_df_unspliced = df.copy()

In [4]:
_df_unspliced = pd.read_pickle(UNSPLICED_RESULT_PATH)

_df_unspliced

,Gene stable ID,Gene name,Sequence
0,ENSMUSG00000064336,mt-Tf,GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACA
1,ENSMUSG00000064337,mt-Rnr1,AAAGGTTTGGTCCTGGCCTTATAATTAATTAGAGGTAAAATTACACATGCAAACCTCCATAGACCGGTGTAAAATCCCTTAAACATTTACTTAAAA...
2,ENSMUSG00000064338,mt-Tv,CATAGTGTAGCTTAATATTAAAGCATCTGGCCTACACCCAGAAGATTTCATGACCAATGAACACTCTGA
3,ENSMUSG00000064339,mt-Rnr2,ACTAATCCTAGCCCTAGCCCTACACAAATATAATTATACTATTATATAAATCAAAACATTTATCCTACTAAAAGTATTGGAGAAAGAAATTCGTAC...
4,ENSMUSG00000064340,mt-Tl1,ATTAGGGTGGCAGAGCCAGGAAATTGCGTAAGACTTAAAACCTTGTTCCCAGAGGTTCAAATCCTCTCCCTAATA
5,ENSMUSG00000064341,mt-Nd1,GTGTTCTTTATTAATATCCTAACACTCCTCGTCCCCATTCTAATCGCCATAGCCTTCCTAACATTAGTAGAACGCAAAATCTTAGGGTACATACAA...
6,ENSMUSG00000064342,mt-Ti,AGAAATATGTCTGATAAAAGAATTACTTTGATAGAGTAAATTATAGAGGTTCAAGCCCTCTTATTTCTA
7,ENSMUSG00000064343,mt-Tq,TAGGATAAGGTGTTTAGGTAGCACGGAGAATTTTGAATTCTTAAGTGTAGGTTCAATTCCTATTGTCCTAG
8,ENSMUSG00000064344,mt-Tm,AGTAAGGTCAGCTAATTAAGCTATCGGGCCCATACCCCGAAAACGTTGGTTTAAATCCTTCCCGTACTA
9,ENSMUSG00000064345,mt-Nd2,ATAAATCCTATCACCCTTGCCATCATCTACTTCACAATCTTCTTAGGTCCTGTAATCACAATATCCAGCACCAACCTAATACTAATATGAGTAGGC...


In [5]:
_df_unspliced['Sequence'].str.len().max()

2960899

In [6]:
records = []
for datum in SeqIO.parse(EXON_PATH, "fasta"):
    parts = datum.id.split('|')
    assert len(parts) == 2
    parts.append(str(datum.seq))
    records.append(parts)

df = pd.DataFrame(records, columns=['Gene stable ID', 'Exon stable ID', 'Sequence'])

df = _df_features.merge(df, how='outer')

assert not df['Exon stable ID'].duplicated().any()

df.to_pickle(EXON_RESULT_PATH)

_df_exon = df.copy()

In [7]:
_df_exon = pd.read_pickle(EXON_RESULT_PATH)

_df_exon

,Gene stable ID,Gene name,Exon stable ID,Sequence
0,ENSMUSG00000064336,mt-Tf,ENSMUSE00000521514,GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACA
1,ENSMUSG00000064337,mt-Rnr1,ENSMUSE00000521515,AAAGGTTTGGTCCTGGCCTTATAATTAATTAGAGGTAAAATTACACATGCAAACCTCCATAGACCGGTGTAAAATCCCTTAAACATTTACTTAAAA...
2,ENSMUSG00000064338,mt-Tv,ENSMUSE00000521516,CATAGTGTAGCTTAATATTAAAGCATCTGGCCTACACCCAGAAGATTTCATGACCAATGAACACTCTGA
3,ENSMUSG00000064339,mt-Rnr2,ENSMUSE00000521517,ACTAATCCTAGCCCTAGCCCTACACAAATATAATTATACTATTATATAAATCAAAACATTTATCCTACTAAAAGTATTGGAGAAAGAAATTCGTAC...
4,ENSMUSG00000064340,mt-Tl1,ENSMUSE00000521518,ATTAGGGTGGCAGAGCCAGGAAATTGCGTAAGACTTAAAACCTTGTTCCCAGAGGTTCAAATCCTCTCCCTAATA
5,ENSMUSG00000064341,mt-Nd1,ENSMUSE00000521519,GTGTTCTTTATTAATATCCTAACACTCCTCGTCCCCATTCTAATCGCCATAGCCTTCCTAACATTAGTAGAACGCAAAATCTTAGGGTACATACAA...
6,ENSMUSG00000064342,mt-Ti,ENSMUSE00000521520,AGAAATATGTCTGATAAAAGAATTACTTTGATAGAGTAAATTATAGAGGTTCAAGCCCTCTTATTTCTA
7,ENSMUSG00000064343,mt-Tq,ENSMUSE00000521521,TAGGATAAGGTGTTTAGGTAGCACGGAGAATTTTGAATTCTTAAGTGTAGGTTCAATTCCTATTGTCCTAG
8,ENSMUSG00000064344,mt-Tm,ENSMUSE00000521522,AGTAAGGTCAGCTAATTAAGCTATCGGGCCCATACCCCGAAAACGTTGGTTTAAATCCTTCCCGTACTA
9,ENSMUSG00000064345,mt-Nd2,ENSMUSE00000521523,ATAAATCCTATCACCCTTGCCATCATCTACTTCACAATCTTCTTAGGTCCTGTAATCACAATATCCAGCACCAACCTAATACTAATATGAGTAGGC...


In [8]:
df = pd.read_csv(FEATURE_PATH, sep='\t')

df = df.groupby(['Gene Synonym']).filter(lambda x: x['Gene name'].dropna().drop_duplicates().shape[0] == 1)

df = df[['Gene Synonym', 'Gene name']]

df = df.drop_duplicates()

df = df.set_index(['Gene Synonym'])

ds = df['Gene name']

gene_synonym_gene_name = ds.to_dict()

In [9]:
records = []
for data in SeqIO.parse(PROTEIN_FASTA_PATH, "fasta"):
    record={}
    for index, value in enumerate(re.findall(r'(?:(?<=^)[^|]+|(?<=\|)[^| ]+|(?<=\|)[^ ]+)|(?<=\ ).+?(?= \w\w\=|$)', data.description)):
        value = value.strip()
        if index == 0:
            record['db'] = value
        elif index == 1:
            record['UniqueIdentifier'] = value
        elif index == 2:
            record['EntryName'] = value
        elif index == 3:
            record['ProteinName'] = value
        else:
            k, v = re.split(r'\=', value)
            record[k] = v

    record['seq'] = str(data.seq)
    
    records.append(record)

In [10]:
df = pd.DataFrame(records)

df['gene_symbol_harmonized'] = df['GN'].apply(lambda x: gene_synonym_gene_name.get(x, x))

df = df.set_index('gene_symbol_harmonized').reset_index()

_df_protein_sequence = df.copy()

In [11]:
df = _df_protein_sequence.copy()

df = df[['gene_symbol_harmonized', 'db', 'UniqueIdentifier', 'EntryName', 'ProteinName', 'OS', 'OX', 'GN', 'PE', 'SV', 'seq']]

assert not df['UniqueIdentifier'].duplicated().any()

df.to_pickle(PROTEIN_RESULT_PATH)